<a href="https://colab.research.google.com/github/nonoumasy/LA-County-Covid-Cases/blob/master/LA_County_Covid_Cases_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from geopy.geocoders import Nominatim

In [0]:
# load data from url
df = pd.DataFrame(pd.read_html('http://publichealth.lacounty.gov/media/Coronavirus/locations.htm')[0])

In [0]:
# skip rows
la_df = df.iloc[22:, :].reset_index(drop=True)

# long beach and pasadena
lb_df = df.iloc[2:4, :]

# number of cases and deaths
cases_deaths_df = df.iloc[[0, 5], :]
cases_deaths_df.columns = [0,1]
cases_deaths_df.to_csv('death.csv')

In [0]:
def cleanup_df(df):
  df.columns = df.columns.str.lower().str.replace(' ', '_')

  df = df[df.locations != 'City of Los Angeles']

  # cleanup values
  df.locations = df.locations.str.replace('*', '').str.replace('City of ', '').str.replace('Los Angeles - ', '').str.replace('Unincorporated - ', '').str.replace('-', '').str.strip()
  df['total_cases'] = df.total_cases.str.replace('--', '0')

  # convert to numeric column
  df['total_cases'] = pd.to_numeric(df['total_cases'])

  # drop under investigation
  df = df[df.locations != 'Under Investigation']

  # split location and keep first value
  df['locations'] = df['locations'].apply(lambda x : x.split('/')[0])

  #
  df = df.sort_values('locations')

  # reset index
  df.reset_index(drop=True, inplace=True)

  return df

In [0]:
la_df = cleanup_df(la_df)
lb_df = cleanup_df(lb_df)

In [0]:
#
la_df.drop_duplicates('locations', inplace=True)
new_df = pd.concat([la_df, lb_df], ignore_index=True)
new_df.drop_duplicates('locations', keep='last', inplace=True)
new_df = new_df.reset_index()

# load locations_df with geocoded
locations_df = pd.read_csv('/content/locations_df.csv')

# merge
temp_df = new_df.merge(locations_df)

# save to csv
temp_df[['locations', 'total_cases', 'lat', 'lon']].to_csv('la_county_coronavirus.csv', )

# Geocode

In [0]:
# prep location for geocoding. Explicitly add ', Los Angeles, California' so the geocoder finds the location

def geocode(df):
  df['loc_prep'] = 'temp'

  for i in range(len(df)):
    df['loc_prep'][i] = str(df.ocations[i]) + ', Los Angeles, California'

  # create and initialize coords column
  df['coords'] = 0.0

  # geocode locations
  geolocator = Nominatim(user_agent="sdfasdf")

  # to space out the geocoding
  from geopy.extra.rate_limiter import RateLimiter

  geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
  df['coords'] = df.loc_prep.apply(lambda x : geocode(x))

  #
  df['lat'] = df.coords.apply(lambda x: x.latitude if x !=None else 0.0)
  df['lon'] = df.coords.apply(lambda x: x.longitude if x !=None else 0.0)

  return df